In [94]:
import numpy as np 
import pandas as pd 	
import matplotlib.pyplot as plt 
import math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from fractions import Fraction

# Naive Bayes - From Scratch

We can expand on the the idea of Bayesean updates with multiple features into a full predictive model. 

## Parts of the Model

Our predictive model needs to do a few things. The main conceptual difference between this model and doing the Bayes tables is that here we will pre-calculate all of the potential calculations of likelihood that we may need. In the tables, every time we saw a new feature we then calculated its likelihood and updated our probability with it when we did the update to generate the unnormalized and posterior probabilities. Here we will pre-calculate each of those likelihoods ahead of time in the fitting step, so when a prediction needs to be made we can just look up the matching likelihoods and do the multiplication to calculate the answer. Doing it like this means all of the heavy lifting (calculating all of the probabilities) is done while fitting, and creating predictions is fast. 

### Initialization

The initialization step will just setup the pieces that we'll need. Here our initialization declares empty varaibles that will hold everything we calculate and figure out. 

### Train the Model

The training process is where the model is "built", or where it learns all of the information it needs to be able to make predictions. This is the majority of the work. In short, we need to:
<ul>
<li> Create a list of all the features in our data. 
<li> Create a list of all of the likelihood possibilities. 
    <ul>
    <li> This is a list of all the potential feature_outcome pairs that is possible. 
    <li> 
    <li> The format, with an underscore between the feature and value is arbitrary - we are only creating a replicable item to represent that combination, it could theoretically be anything. 
    </ul>
<li> Calculate the actual likelihood probabilities. 
    <ul>
    <li> For each of the possible likelihoods, calculate its actual likelihood value. 
    <li> I.e. what is the actual probability of playing golf given that it is sunny.
    <li> This is the "flipped conditional" part of the Bayes equation. 
    </ul>
<li> Calculate class prior probabilities. 
    <ul>
    <li> This is the other part of the numerator in the Bayes equation. 
    </ul>
<li> Calculate the prior probabilities for the features. 
    <ul>
    <li> How likely is each potential value for each feature. 
    <li> Used for the bottom of the Bayes equation. 
    </ul>
</ul>

Once done training, no matter what our data actually says, we should have all of the probabilities for it pre-calculated in one of our data structures. 

### Predict

To make a prediction we just need to look up the correct probabilities, and perform the calculation. 

### Important Bayes Note

One problem that can occur with Bayes is the "zero count" problem, or what if we have a valid value for one of our features that just doesn't occur in our training data. For example, what if there was a day that we were predicting with our weather and golf model where the Humidity was "Dry"? When we attempt to look up the probability of "Humidity = Dry" there won't be anything there and the probability will be 0, since we didn't train for it. Since this will popup on the bottom of the division in the equation, that'll be an issue. This problem is resolved through something called Laplace (or Additive) Smoothing. We won't implement this immediately here (maybe next week? maybe for a fun weekend exercise?) but the idea is relatively simple - when calculating the probability of something, rather than doing the normal calculation of:

$ P(A) = \frac{count-of-A}{total-elements} $

We change that to:

$ P(A) = \frac{count-of-A + alpha}{total-elements + (alpha*number-of-features)} $

Where alpha is a chosen constant, usually 1. We'll look at how to choose a good alpha when we look at tuning models, the short answer is guess and test. 

This smoothing correction seves to make sure that our model can handle new values without just failing, at the expense of a very minor impact to accuracy on predicted probabilities for things we do know. If a new unseen value comes in, rather than it's probability being 0, which will cause the overall cacluation to fail, its probability will be some small value - almost certainly unlikely to make a tangible difference in our calculations, but enough to keep things rolling. In other words we've sacrificed a tiny bit of accuracy in exchange for much more generalizability. With datasets of a reasonable size, the small additions of the smoothing calculation don't make much of a difference. 

In a perfect world, where we knew every possibility ahead of time and had it embedded in our training data, this smoothing would not be useful and we would not consider it. In reality, Bayes is often used for things like spam detection, where the incoming features are words from the text of an email - in such a case, it is very likely that we will encounter new things when making predictions, so this smoothing is used very frequently when using a Naive Bayes model. 


In [98]:
def update(table):
    """Compute the posterior probabilities."""
    table['unnorm'] = table['prior'] * table['likelihood']
    prob_data = table['unnorm'].sum()
    table['posterior'] = table['unnorm'] / prob_data
    return prob_data

In [90]:
df = pd.read_table("data/weather.txt")
df.head()

,Outlook,Temp,Humidity,Windy,Play
0,Rainy,Hot,High,f,no
1,Rainy,Hot,High,t,no
2,Overcast,Hot,High,f,yes
3,Sunny,Mild,High,f,yes
4,Sunny,Cool,Normal,f,yes


### Updates

We can do a warm-up to making our model by making an update table simplified version. 

What is the probability of playing when the weather is Sunny, Hot, Normal, False

In [163]:
weather = pd.DataFrame(index=["Play", "Not Play"])
total = len(df)
dfPlay = df[df["Play"] == "yes"]
dfNoPlay = df[df["Play"] == "no"]
pPlay = Fraction(len(df[df["Play"] == "yes"]), total)
weather["prior"] = pPlay, (1 - pPlay)
updates = 1
weather

,prior
Play,9/14
Not Play,5/14


In [164]:
if_playOut = Fraction(len(dfPlay[dfPlay["Outlook"] == "Sunny"]), len(dfPlay))
if_notOut = Fraction(len(dfNoPlay[dfNoPlay["Outlook"] == "Sunny"]), len(dfNoPlay))
weather["likelihood"] = if_playOut, if_notOut
update(weather)
weather

,prior,likelihood,unnorm,posterior
Play,9/14,1/3,3/14,3/5
Not Play,5/14,2/5,1/7,2/5


#### Update Table

We'll tweak the prevous update steps to handle multiple rounds. This isn't the most efficient way to do this, we're digging through the details here. 

We will run the updates just as we did before, the change here is each time I'll rename the old prior and likelihood values, so we keep everything as we go though. 

In [165]:
weather = weather.rename(columns={"prior": str("prior"+str(updates)), "unnorm": "prior", "likelihood":("likelihood"+str(updates))})
weather = weather.drop(columns={"posterior"})
updates += 1
weather

,prior1,likelihood1,prior
Play,9/14,1/3,3/14
Not Play,5/14,2/5,1/7


In [166]:
if_playTemp = Fraction(len(dfPlay[dfPlay["Temp"] == "Hot"]), len(dfPlay))
if_notTemp = Fraction(len(dfNoPlay[dfNoPlay["Temp"] == "Hot"]), len(dfNoPlay))
weather["likelihood"] = if_playTemp, if_notTemp
update(weather)
weather

,prior1,likelihood1,prior,likelihood,unnorm,posterior
Play,9/14,1/3,3/14,2/9,1/21,5/11
Not Play,5/14,2/5,1/7,2/5,2/35,6/11


In [167]:
weather = weather.rename(columns={"prior": str("prior"+str(updates)), "unnorm": "prior", "likelihood":("likelihood"+str(updates))})
weather = weather.drop(columns={"posterior"})
updates += 1
weather

,prior1,likelihood1,prior2,likelihood2,prior
Play,9/14,1/3,3/14,2/9,1/21
Not Play,5/14,2/5,1/7,2/5,2/35


In [168]:
if_playHum2 = Fraction(len(dfPlay[dfPlay["Humidity"] == "Normal"]) , len(dfPlay))
if_notHum2 = Fraction(len(dfNoPlay[dfNoPlay["Humidity"] == "Normal"]) , len(dfNoPlay))
weather["likelihood"] = if_playHum2, if_notHum2
update(weather)
#weather
weather = weather.rename(columns={"prior": str("prior"+str(updates)), "unnorm": "prior", "likelihood":("likelihood"+str(updates))})
weather = weather.drop(columns={"posterior"})
updates += 1
weather

,prior1,likelihood1,prior2,likelihood2,prior3,likelihood3,prior
Play,9/14,1/3,3/14,2/9,1/21,2/3,2/63
Not Play,5/14,2/5,1/7,2/5,2/35,1/5,2/175


#### Last Update Step

After this update I'll keep the posterior probability, since we are done, and I'll drop all the interim prior probabilities since we have no use for them. 

In [169]:
if_playWind2 = Fraction(len(dfPlay[dfPlay["Windy"] == "f"]) , len(dfPlay))
if_notWind2 = Fraction(len(dfNoPlay[dfNoPlay["Windy"] == "f"]) , len(dfNoPlay))
weather["likelihood"] = if_playWind2, if_notWind2
update(weather)
weather.drop(columns={"prior2", "prior", "prior3"}, inplace=True)
weather

,prior1,likelihood1,likelihood2,likelihood3,likelihood,unnorm,posterior
Play,9/14,1/3,2/9,2/3,2/3,4/189,125/152
Not Play,5/14,2/5,2/5,1/5,2/5,4/875,27/152


In [175]:
# Self Check
# Play unnormalized probabilities
print(Fraction(9,14)*Fraction(1,3)*Fraction(2,9)*Fraction(2,3)*Fraction(2,3))

4/189


### Results

What we are left with here is a series of likelihoods that each modify our prior probability. If we're keen, we also notice that this series of likelihoods translates directly to the top part of the Bayes equation. The bottom bit is always the same, and we only need that to normalize, so the result of the prediction is whichever unnorm probability is higher. Here 4/189 is more likely than 4/875, so we predict we'll play. 

This is all our classifier needs to do! We just take the prior probabilities and all the likelihoods, multiply them through, and pick the most likely outcome!! This should be pretty easy to put into place:
<ul>
<li> The training part of the model can just calculate all of these likelihoods and prior probabilities. Each one was just simple math, so we'll calculate all of them and store them in some kind of list (a dictionary, actually).
    <ul>
    <li> E.g. the likelihood of playing golf if it is sunny can be calculated and saved, same with the prob of not playing if it is windy, etc...
    </ul>
<li> The predicting part is just looking up the prior and the matching likelihoods from our dictionary of precalculated values, and doing the math. 
</ul>

We are awesome!

### Building Naive Bayes

Below we have most of the Naive Bayes algorithm. The "real" ones from sklearn or other packages are basically the same as this - normally just with slightly better error handling, a few more options, and maybe some optimizations for speed - but the idea and execution is almost the same. One change that is actually different in ours is that we are using data in its dataframe format within our algorithm. This is a relatively minor change, and doesn't change any of the concepts behind what we are doing, what it does is make the code a little easier to read and understand as humans, as we can use things like the dataframe's "columns" variable. An implementation using arrays (to work exactly like the sklearn ones) would be almost the same, but places where we refer to columns by names would be replaced by indicies. 

We can use some print statements to look at exactly what is going on inside our model. This is also a good exercise as if we can't figure something out, printing the current state is the easiest way to diagnose it. 

In [170]:
class  NaiveBayes(object):
	def __init__(self):

		"""
			Attributes:
				likelihoods: Likelihood of each feature per class
				class_priors: Prior probabilities of classes 
				pred_priors: Prior probabilities of features 
				features: All features of dataset

		"""
		self.features = list
		self.likelihoods = {} # All of the possible feature-value pairs
		self.class_priors = {} # All class priors
		self.pred_priors = {} # All the predictor priors (for the bottom)

		self.X_train = np.array
		self.y_train = np.array
		self.train_size = int
		self.num_feats = int

	def fit(self, X, y):

		self.features = list(X.columns)
		self.X_train = X
		self.y_train = y
		self.train_size = X.shape[0]
		self.num_feats = X.shape[1]

		# Generate a list of all the possible "likelihoods"
		# Each one is a feature_outcome pair, e.g. from the golf
		# one: Outlook_sunny, Temperature_hot, etc...
		for feature in self.features:
			# Initialize the list of possible values for each feature in the big dictionary of features. 
			self.likelihoods[feature] = {}
			self.pred_priors[feature] = {}

			# Loop through all of the values that this feature can take on, and add that
			# as a possibility to the list of likelihoods for that feature. 
			for feat_val in np.unique(self.X_train[feature]):
				self.pred_priors[feature].update({feat_val: 0})
				#print(feat_val)
				for outcome in np.unique(self.y_train):
					self.likelihoods[feature].update({feat_val+'_'+outcome:0})
					self.class_priors.update({outcome: 0})
					#print('\t'+feat_val+'_'+outcome)
		
		# These functions build the rest of the "learned knowledge"
		# of the model. 
		# Calculate the priors 
		self._calc_class_prior()
		# Calculate the probability of each likelihood
		# We precalculate and save each one so when predictions come
		# we can just look up the probabilities and multiply. 
		self._calc_likelihoods()
		# Generate prior probs for each feature
		self._calc_predictor_prior()

	def _calc_class_prior(self):

		""" P(c) - Prior Class Probability """

		for outcome in np.unique(self.y_train):
			outcome_count = sum(self.y_train == outcome)
			self.class_priors[outcome] = outcome_count / self.train_size
			#print(outcome, outcome_count, self.class_priors[outcome])

	def _calc_likelihoods(self):

		""" P(x|c) - Likelihood """

		for feature in self.features:
			print("\n Likelihoods for:", feature)
			for outcome in np.unique(self.y_train):
				outcome_count = sum(self.y_train == outcome)
				feat_likelihood = self.X_train[feature][self.y_train[self.y_train == outcome].index.values.tolist()].value_counts().to_dict()
				print("Outcome:", outcome)

				for feat_val, count in feat_likelihood.items():
					self.likelihoods[feature][feat_val + '_' + outcome] = count/outcome_count
					print(self.likelihoods[feature], self.likelihoods[feature][feat_val + '_' + outcome])


	def _calc_predictor_prior(self):

		""" P(x) - Evidence """

		for feature in self.features:
			feat_vals = self.X_train[feature].value_counts().to_dict()
			#print(feat_vals)

			for feat_val, count in feat_vals.items():
				self.pred_priors[feature][feat_val] = count/self.train_size
				#print(self.pred_priors[feature][feat_val])


	def predict(self, X):

		""" Calculates Posterior probability P(c|x) """
		# Make predictions:
		# look up each likelihood, multiply out the likelihood
		# normalize and make a prediction of the highest. 
		results = []
		X = np.array(X)
		print("\n Predictions:")
		# Loop through the things that we are predicting. 
		# Each query is one record from X, our dataset of features. 
		for query in X:
			probs_outcome = {}
			for outcome in np.unique(self.y_train):
				# Get the prior probability of the outcome
				prior = self.class_priors[outcome]
				likelihood = 1
				evidence = 1

				# Loop through each feature, and its value. 
				# get the likelihood for that feature, 
				# update the running probability with that likelihood. 
				for feat, feat_val in zip(self.features, query):
					likelihood *= self.likelihoods[feat][feat_val + '_' + outcome]
					evidence *= self.pred_priors[feat][feat_val]

				# This is the Bayes final calculation
				posterior = (likelihood * prior) / (evidence)
				# store the postirior probability in the array of the results. 
				# each row of data we are predicting gets a prediction, so we end up with a bunch. 
				probs_outcome[outcome] = posterior

			# Translate the posterior probs to a prediction i.e. what is the actual predicted class. 
			# The prediction is whichever outcome is most likely. 
			result = max(probs_outcome, key = lambda x: probs_outcome[x])
			results.append(result)
		# Return an array of all the predictions, just like we are used to getting from any model
		return np.array(results)


In [171]:
def pre_processing(df):

	""" partioning data into features and target """

	X = df.drop([df.columns[-1]], axis = 1)
	y = df[df.columns[-1]]

	return X, y

## Check the Model

In [172]:
#Weather Dataset
print("\nWeather Dataset:")


#print(df)

#Split fearures and target
X,y  = pre_processing(df)

nb_clf = NaiveBayes()
nb_clf.fit(X, y)

print("Train Accuracy: {}".format(accuracy_score(y, nb_clf.predict(X))))

#Query 1:
query = np.array([['Rainy','Mild', 'Normal', 't']])
print("Query 1:- {} ---> {}".format(query, nb_clf.predict(query)))

#Query 2:
query = np.array([['Overcast','Cool', 'Normal', 't']])
print("Query 2:- {} ---> {}".format(query, nb_clf.predict(query)))

#Query 3:
query = np.array([['Sunny','Hot', 'High', 't']])
print("Query 3:- {} ---> {}".format(query, nb_clf.predict(query)))


Weather Dataset:

 Likelihoods for: Outlook
Outcome: no
{'Overcast_no': 0, 'Overcast_yes': 0, 'Rainy_no': 0.6, 'Rainy_yes': 0, 'Sunny_no': 0, 'Sunny_yes': 0} 0.6
{'Overcast_no': 0, 'Overcast_yes': 0, 'Rainy_no': 0.6, 'Rainy_yes': 0, 'Sunny_no': 0.4, 'Sunny_yes': 0} 0.4
Outcome: yes
{'Overcast_no': 0, 'Overcast_yes': 0.4444444444444444, 'Rainy_no': 0.6, 'Rainy_yes': 0, 'Sunny_no': 0.4, 'Sunny_yes': 0} 0.4444444444444444
{'Overcast_no': 0, 'Overcast_yes': 0.4444444444444444, 'Rainy_no': 0.6, 'Rainy_yes': 0, 'Sunny_no': 0.4, 'Sunny_yes': 0.3333333333333333} 0.3333333333333333
{'Overcast_no': 0, 'Overcast_yes': 0.4444444444444444, 'Rainy_no': 0.6, 'Rainy_yes': 0.2222222222222222, 'Sunny_no': 0.4, 'Sunny_yes': 0.3333333333333333} 0.2222222222222222

 Likelihoods for: Temp
Outcome: no
{'Cool_no': 0, 'Cool_yes': 0, 'Hot_no': 0.4, 'Hot_yes': 0, 'Mild_no': 0, 'Mild_yes': 0} 0.4
{'Cool_no': 0, 'Cool_yes': 0, 'Hot_no': 0.4, 'Hot_yes': 0, 'Mild_no': 0.4, 'Mild_yes': 0} 0.4
{'Cool_no': 0.2, 'Cool_

## Use the Model